In [56]:
import pandas as pd
from pandas import DataFrame
import random
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from pyvis.network import Network
import community.community_louvain as community_louvain
from tqdm import tqdm

---

In [2]:
# TODO : 
# Faire fonction de vérification des paires id: community pour le dataframe
# Revoir la méthode de sauvegarde des graphes (qui des anciens graphs)
# 

## Importation des données

In [58]:
project = pd.read_csv("../Data/CSV/Projects/project.csv", sep=";")

In [4]:
organization = pd.read_csv("../Data/CSV/Projects/organization.csv", sep=";")

On importera les pubications après

In [5]:
print(project.shape)
print(organization.shape)

(35381, 20)
(177190, 25)


---

## Création du graph

On récupère les ID unique des projets

In [6]:
project_id = list(project["id"].unique())

Pour chaque projet je récupère les id des organisations qui y participent. Tout ça dans un dictionnaire.

In [7]:
projet_orga = {}
for projet in project_id:
    projet_orga[projet] = list(organization[organization["projectID"] == projet]["organisationID"])

Création d'un graph vide

In [8]:
G = nx.Graph()

Fonction de création du graph (prendre plus d'une heure à faire tourner)

In [9]:
########### /!\ TOURNE INDEFINIEMENT /!\ ##############

yes_counter = 0
for i, tq in zip(range(1, len(projet_orga.keys())), tqdm(range(len(projet_orga.keys())))):
#     print(projet_orga[project_id[i]])
    for t in range(len(projet_orga.keys())):
#         G.add_node(project_id[i - 1])
        G.add_node(project_id[i - 1], label=project_id[i - 1])
        for y in projet_orga[project_id[i - 1]]:
#         print(f"i: {i} | i+1: {projet_orga[project_id[i+1]]} | y: {y}")
            if G.has_edge(project_id[i - 1], project_id[t]) is False:
                for x in projet_orga[project_id[t]]:
            
                    if y == x:
                        yes_counter += 1
                        G.add_edge(project_id[i-1], project_id[t])

G.remove_edges_from(nx.selfloop_edges(G))
# On supprime les boucles du graph car cela n'apporte pas d'information et cela prend de la place

  0%|                                                                              | 8/35381 [00:02<3:31:10,  2.79it/s]


KeyboardInterrupt: 

Pour importer un graph sauvegardé au format .pickle

In [18]:
 G = pickle.load(open("../graphs_save/graphe_3_labeled.pickle", "rb"))

In [19]:
G.number_of_nodes()

35381

In [20]:
G.number_of_edges()

9982209

---

## Détection de communauté

### Détection de communauté

In [21]:
partition = community_louvain.best_partition(G)

In [22]:
print(f"Nombre de communautés : {len(set(partition.values()))}")

Nombre de communautés : 4381


On ajoute les communauté en tant qu'attributs aux noeuds du graph

In [23]:
nx.set_node_attributes(G, partition, 'partition')

On ajoute les communautés au DataFrame de données

In [59]:
def dict_to_df(dict_: dict, df: DataFrame, name: str, dtype: str = "int32", join_on: str = "id") -> DataFrame:

    dict_df = pd.DataFrame.from_dict(dict_, orient="index").astype({0:dtype})
    df = df.join(dict_df, on=join_on)
    df.rename({0:name}, axis=1, inplace=True)
    
    return df

In [61]:
project = dict_to_df(partition, project, "Community")

In [27]:
project.head(2)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi,Community
0,817296,eDCaseMAN,CLOSED,"Defendable legal cases through an affordable, ...",2018-05-01,2018-09-30,71429,50000,H2020-EU.2.3.,EIC-SMEInst-2018-2020,...,H2020,H2020-EIC-SMEInst-2018-2020,H2020-SMEInst-2018-2020-1,SME-1,NaN,"""Electronic discovery (known as e-discovery) i...",2022-08-10 13:07:21,217713,10.3030/817296,0
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,H2020,H2020-LCE-2016-2017,H2020-LCE-2017-RES-CSA,CSA,NaN,The overall objective of WinWind is to enhance...,2022-08-17 10:54:17,211548,10.3030/764717,1


### Modification et analyse des communautés

On calcule la proportion du chaque communauté par rapport à tous le data set 

In [75]:
proportions = project['Community'].value_counts(normalize=True).to_frame()
proportions.rename({"Community": "Proportion"}, inplace=True, axis=1)

In [76]:
proportions

,Proportion
1,0.234759
5,0.088720
4,0.086120
13,0.071309
17,0.059269
...,...
1642,0.000028
1643,0.000028
1644,0.000028
1645,0.000028


On recupère l'index (ici les communautés) pour lesquelles la proportion est inférieure à 1% du data set 

In [77]:
index_label = proportions[proportions["Proportion"] <= 0.01].index.to_list()

On modifie la valeur des communautés pour lesquelles la proportion est inférieure à 1%

In [78]:
def community_filter(x: int):
    if x in index_label:
        return -99
    else:
        return x

In [79]:
project["New_Community"] = project["Community"].apply(community_filter)

In [80]:
project.head(3)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi,Community,Degree,Degree_Centrality,New_Community
0,817296,eDCaseMAN,CLOSED,"Defendable legal cases through an affordable, ...",2018-05-01,2018-09-30,71429,50000,H2020-EU.2.3.,EIC-SMEInst-2018-2020,...,SME-1,NaN,"""Electronic discovery (known as e-discovery) i...",2022-08-10 13:07:21,217713,10.3030/817296,0,0,0.000000,-99
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,CSA,NaN,The overall objective of WinWind is to enhance...,2022-08-17 10:54:17,211548,10.3030/764717,1,427,0.012069,1
2,784994,PentaHelix,CLOSED,Multi stakeholder and governance approach for ...,2018-03-01,2021-09-30,"1808343,75","1808343,75",H2020-EU.3.3.,EE-09-2016-2017,...,CSA,NaN,The PentaHelix project is focusing on developi...,2022-04-11 16:42:12,213566,10.3030/784994,1,74,0.002092,1


In [81]:
project.shape

(35381, 24)

On crée un dictionnaire avec l'ID des projets et leurs nouvelles communautés pour être ajouté aux attributs des noeuds du graph

In [82]:
new_community = project.set_index('id')["New_Community"].to_dict()

In [83]:
set(new_community.values())

{-99, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 19}

In [86]:
print(f"Nombre de communautés après traitement : {len(set(new_community.values()))}")

Nombre de communautés après traitement : 17


On ajoute ces nouvelles communautés au graph ! 

In [87]:
# nx.set_node_attributes(G, new_community, 'new_partition')

On enregistre ce nouveau graph

In [88]:
# pickle.dump(G, open('../graphs_save/graph_3_labeled_new_partition.pickle', 'wb'))

In [89]:
# nx.write_gexf(G, "../graphs_save/graph_3_labeled_new_partition.gexf")

On va changer les noms des communautés :

In [90]:
old_names = set(new_community.values())

changing_names = {}
for name in enumerate(old_names):
    changing_names[name[1]] = name[0]

In [91]:
changing_names

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 8: 6,
 10: 7,
 11: 8,
 12: 9,
 13: 10,
 14: 11,
 15: 12,
 16: 13,
 17: 14,
 19: 15,
 -99: 16}

In [92]:
def change_name(x: int):
    return changing_names[x]

In [93]:
project["New_Community_Name"] = project["New_Community"].apply(change_name)

In [94]:
project.head(3)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,nature,objective,contentUpdateDate,rcn,grantDoi,Community,Degree,Degree_Centrality,New_Community,New_Community_Name
0,817296,eDCaseMAN,CLOSED,"Defendable legal cases through an affordable, ...",2018-05-01,2018-09-30,71429,50000,H2020-EU.2.3.,EIC-SMEInst-2018-2020,...,NaN,"""Electronic discovery (known as e-discovery) i...",2022-08-10 13:07:21,217713,10.3030/817296,0,0,0.000000,-99,16
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,NaN,The overall objective of WinWind is to enhance...,2022-08-17 10:54:17,211548,10.3030/764717,1,427,0.012069,1,0
2,784994,PentaHelix,CLOSED,Multi stakeholder and governance approach for ...,2018-03-01,2021-09-30,"1808343,75","1808343,75",H2020-EU.3.3.,EE-09-2016-2017,...,NaN,The PentaHelix project is focusing on developi...,2022-04-11 16:42:12,213566,10.3030/784994,1,74,0.002092,1,0


### Sauvegarde des communautés

Pour sauvegarder le graph au format .pickle

In [35]:
# pickle.dump(G, open('../graphs_save/graphe_3_labeled.pickle', 'wb'))

Pour sauvegarder le graph en format gexf pour l'importer sur Gephi

In [ ]:
# nx.write_gexf(G, "../graphs_save/graphe_3_labeled.gexf")

---

## Calcules sur le graph

In [49]:
degrees = dict(nx.degree(G))

In [63]:
project = dict_to_df(degrees, project, "Degree")

In [64]:
project.head(3)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi,Community,Degree
0,817296,eDCaseMAN,CLOSED,"Defendable legal cases through an affordable, ...",2018-05-01,2018-09-30,71429,50000,H2020-EU.2.3.,EIC-SMEInst-2018-2020,...,H2020-EIC-SMEInst-2018-2020,H2020-SMEInst-2018-2020-1,SME-1,NaN,"""Electronic discovery (known as e-discovery) i...",2022-08-10 13:07:21,217713,10.3030/817296,0,0
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,H2020-LCE-2016-2017,H2020-LCE-2017-RES-CSA,CSA,NaN,The overall objective of WinWind is to enhance...,2022-08-17 10:54:17,211548,10.3030/764717,1,427
2,784994,PentaHelix,CLOSED,Multi stakeholder and governance approach for ...,2018-03-01,2021-09-30,"1808343,75","1808343,75",H2020-EU.3.3.,EE-09-2016-2017,...,H2020-EE-2016-2017,H2020-EE-2017-CSA-PPI,CSA,NaN,The PentaHelix project is focusing on developi...,2022-04-11 16:42:12,213566,10.3030/784994,1,74


In [72]:
degree_centrality = nx.degree_centrality(G)

In [73]:
project = dict_to_df(degree_centrality, project, "Degree_Centrality", dtype="float64")

In [74]:
project.head(2)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi,Community,Degree,Degree_Centrality
0,817296,eDCaseMAN,CLOSED,"Defendable legal cases through an affordable, ...",2018-05-01,2018-09-30,71429,50000,H2020-EU.2.3.,EIC-SMEInst-2018-2020,...,H2020-SMEInst-2018-2020-1,SME-1,NaN,"""Electronic discovery (known as e-discovery) i...",2022-08-10 13:07:21,217713,10.3030/817296,0,0,0.000000
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,H2020-LCE-2017-RES-CSA,CSA,NaN,The overall objective of WinWind is to enhance...,2022-08-17 10:54:17,211548,10.3030/764717,1,427,0.012069


Centralité de proximité : cette méthode calcule la centralité d'un noeud en fonction de la distance moyenne à tous les autres noeuds. Plus un noeud est proche des autres noeuds, plus sa centralité est élevée.

In [117]:
closeness_centrality = nx.closeness_centrality(G)

KeyboardInterrupt: 

Centralité de betweenness : cette méthode calcule la centralité d'un noeud en fonction de sa participation aux chemins les plus courts entre tous les autres noeuds. Plus un noeud est sur des chemins courts entre les autres noeuds, plus sa centralité est élevée.

In [ ]:
betweenness_centrality = nx.betweenness_centrality(G)

Centralité de eigenvector : cette méthode calcule la centralité d'un noeud en fonction de la centralité de ses voisins. Les noeuds connectés à des noeuds centraux auront une centralité plus élevée.

In [118]:
eigenvector_centrality = nx.eigenvector_centrality(G)

KeyboardInterrupt: 

---

## Importation des données de publications

In [95]:
publication_0 = pd.read_csv("../Data/CSV/Publications/projectPublications.csv", sep=";")
publication_1 = pd.read_csv("../Data/CSV/Publications/projectPublications_2.csv", sep=";")
publication_2 = pd.read_csv("../Data/CSV/Publications/projectPublications_3.csv", sep=";")
publication_3 = pd.read_csv("../Data/CSV/Publications/projectPublications_4.csv", sep=";")
publication_4 = pd.read_csv("../Data/CSV/Publications/projectPublications_5.csv", sep=";")
publication_5 = pd.read_csv("../Data/CSV/Publications/projectPublications_6.csv", sep=";")

In [96]:
print(publication_0.shape)
print(publication_1.shape)
print(publication_2.shape)
print(publication_3.shape)
print(publication_4.shape)
print(publication_5.shape)

(55001, 16)
(55001, 16)
(55001, 16)
(55001, 16)
(55001, 16)
(26586, 16)


In [97]:
publications_list = [ publication_0,
publication_1,
publication_2,
publication_3,
publication_4,
publication_5 ]

On fusionne tous les dataframes en un seul

In [98]:
publication = pd.DataFrame()
for publi in publications_list:
    publication = pd.concat([publication, publi], axis=0)

### Informations sur les données

In [99]:
publication.shape

(301591, 16)

In [100]:
publication.head()

,id,title,isPublishedAs,authors,journalTitle,journalNumber,publishedYear,publishedPages,issn,isbn,doi,projectID,projectAcronym,collection,contentUpdateDate,rcn
0,823965_1537977_PUBLI,On the Utility of High-Resolution Soil Moistur...,Peer reviewed articles,"Ait Hssaine, B.; Chehbouni, A.; Er-Raki, S.; K...",Remote Sensing,NaN,2021.0,NaN,2072-4292,NaN,10.3390/rs13040727,823965,ACCWA,Project publication,2022-09-27 18:18:00,872894
1,764958_1538020_PUBLI,Thermosensitive hydration of four acrylamide-b...,Peer reviewed articles,"Patrick K. Quoika, Maren Podewitz, Yin Wang, A...",The Journal of Physical Chemistry B,2,2020.0,9745 - 9756,1520-5207,NaN,10.1021/acs.jpcb.0c07232,764958,NANOSTEM,Project publication,2022-09-27 18:20:01,872958
2,677120_355435_PUBLI,On property (T) for Aut(F_n) and SL_n(Z),Peer reviewed articles,"Marek Kaluba, Dawid Kielak, Piotr W. Nowak",Annals of Mathematics,"Volume 193 (2021), Issue 2",2018.0,539-562,0003-486X,NaN,10.4007/annals.2021.193.2.3,677120,INDEX,Project publication,2022-09-30 20:15:15,544507
3,761042_1539423_PUBLI,Sporulation in solventogenic and acetogenic cl...,Peer reviewed articles,Mamou Diallo; Servé W. M. Kengen; Ana M. López...,Applied Microbiology and Biotechnology,105,2021.0,3533-3557,0175-7598,NaN,10.1007/s00253-021-11289-9,761042,BIOCONCO2,Project publication,2022-09-27 18:19:56,872951
4,871037_1533329_PUBLI,Comprehensive Fragment Screening of the SARS-C...,Peer reviewed articles,"Berg H, Wirtz Martin MA, Altincekic N, Alshaml...",Angew Chem Int Ed Engl.,NaN,2022.0,NaN,1433-7851,NaN,10.1002/anie.202205858,871037,iNEXT-Discovery,Project publication,2022-09-30 20:17:09,870820


### Traitement sur les données

On calcule le nombre de publications pour chaque projectID

In [101]:
print(len(list(publication["projectID"].value_counts().index)))
pub_project = publication["projectID"].value_counts().to_dict()

19395


In [102]:
pub_project

{633053: 6196,
 681202: 1593,
 696656: 942,
 785219: 876,
 785907: 607,
 881603: 554,
 665778: 512,
 669026: 461,
 654109: 441,
 945539: 428,
 654002: 414,
 654148: 414,
 739551: 409,
 739508: 379,
 692419: 379,
 654305: 378,
 690575: 376,
 643417: 361,
 641816: 351,
 689443: 327,
 789028: 307,
 668995: 305,
 870628: 304,
 761727: 304,
 727872: 295,
 830927: 293,
 654360: 289,
 823717: 279,
 727987: 275,
 663830: 269,
 720270: 267,
 633531: 266,
 654208: 263,
 681228: 251,
 730960: 249,
 654024: 248,
 643476: 243,
 678024: 240,
 692145: 239,
 641122: 238,
 730872: 231,
 664931: 231,
 688995: 230,
 824093: 223,
 633595: 220,
 739593: 218,
 692146: 218,
 691135: 216,
 739582: 215,
 694097: 214,
 713279: 213,
 871149: 211,
 691818: 210,
 713673: 210,
 653706: 207,
 777826: 207,
 692194: 206,
 734922: 205,
 690904: 203,
 691714: 202,
 654168: 201,
 692022: 200,
 712949: 200,
 666881: 199,
 668981: 199,
 733206: 199,
 952215: 198,
 678760: 195,
 713683: 193,
 810980: 192,
 741520: 191,
 817

In [103]:
id_project_publi = list(pub_project.keys())
print(len(id_project_publi))

19395


In [139]:
project[project["id"].isin(id_project_publi)].head(3)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,contentUpdateDate,rcn,grantDoi,Community,Degree,Degree_Centrality,New_Community,New_Community_Name,Nb_Publication,Project_Duration
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,2022-08-17 10:54:17,211548,10.3030/764717,1,427,0.012069,1,0,4,912.0
3,841546,RES,TERMINATED,Rights for Ecosystem Services (RES): a framewo...,2020-01-13,2022-01-12,"224933,76","224933,76",H2020-EU.1.3.,MSCA-IF-2018,...,2021-12-30 09:44:32,222283,10.3030/841546,1,141,0.003985,1,0,12,730.0
4,716923,LEVIATHAN,CLOSED,Taming the Leviathan? Legal and Political Acco...,2017-08-01,2022-01-31,1184595,1184595,H2020-EU.1.1.,ERC-2016-STG,...,2022-09-08 18:14:15,206991,10.3030/716923,10,9,0.000254,10,7,12,1644.0


Nous n'avons pas tous les ID des projets car ne sont renseignés que les projets ayant publiés. 

In [105]:
def fill_publication(publi: dict, project_id: list):

    for project in project_id:
        if project in list(publi.keys()):
            continue
        else: 
            publi[project] = 0


In [106]:
fill_publication(pub_project, project_id)

La fonction ressortait un ID supplémentaire par rapport au nombre d'ID de projets qui nous avions. <br>
Voici un petit moyent de trouver quel ID est en trop et le supprimer

In [108]:
# Trouver l'ID du projet en trop
count = 0
for intru in list(pub_project.keys()):
    if intru in project_id:
        continue
#         count += 1
    else: 
        to_delete = intru
        print(to_delete)

743165


In [140]:
# Suppression de ce projet

# del pub_project[743165]
del pub_project[to_delete]

KeyError: 743165

On va ajouter le nombre de publications par projet

In [76]:
# Quid d'avoir le nomnbre de publications pour faire le graphe ? Cela pourrait-il influer négativemment sur la pertinence
# de notre étude économétrique ? 

On va l'ajouter à notre DataFrame

In [111]:
project = dict_to_df(pub_project, project, "Nb_Publication")

In [121]:
# project.to_csv("../Data/CSV/Projets_updated.csv", index=False)

In [112]:
project.head(3)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,objective,contentUpdateDate,rcn,grantDoi,Community,Degree,Degree_Centrality,New_Community,New_Community_Name,Nb_Publication
0,817296,eDCaseMAN,CLOSED,"Defendable legal cases through an affordable, ...",2018-05-01,2018-09-30,71429,50000,H2020-EU.2.3.,EIC-SMEInst-2018-2020,...,"""Electronic discovery (known as e-discovery) i...",2022-08-10 13:07:21,217713,10.3030/817296,0,0,0.000000,-99,16,0
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,The overall objective of WinWind is to enhance...,2022-08-17 10:54:17,211548,10.3030/764717,1,427,0.012069,1,0,4
2,784994,PentaHelix,CLOSED,Multi stakeholder and governance approach for ...,2018-03-01,2021-09-30,"1808343,75","1808343,75",H2020-EU.3.3.,EE-09-2016-2017,...,The PentaHelix project is focusing on developi...,2022-04-11 16:42:12,213566,10.3030/784994,1,74,0.002092,1,0,0


---

## Traitement sur le DataFrame | Feature Engineering

On va calculer le temps entres la date de début et de fin du projet

In [137]:
project["Project_Duration"] = (pd.to_datetime(project["endDate"])) - (pd.to_datetime(project["startDate"]))
project["Project_Duration"] = pd.to_numeric(project["Project_Duration"].dt.days, downcast="integer")
# project["Project_Duration"] = project["Project_Duration"].astype("int16")

In [138]:
project.head(2)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,contentUpdateDate,rcn,grantDoi,Community,Degree,Degree_Centrality,New_Community,New_Community_Name,Nb_Publication,Project_Duration
0,817296,eDCaseMAN,CLOSED,"Defendable legal cases through an affordable, ...",2018-05-01,2018-09-30,71429,50000,H2020-EU.2.3.,EIC-SMEInst-2018-2020,...,2022-08-10 13:07:21,217713,10.3030/817296,0,0,0.000000,-99,16,0,152.0
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,2022-08-17 10:54:17,211548,10.3030/764717,1,427,0.012069,1,0,4,912.0


On va compter le nombre d'organisations par projets

In [166]:
print(len(list(organization["projectID"].value_counts().index)))
orgas_dict = organization["projectID"].value_counts().to_dict()

35381


In [167]:
project = dict_to_df(orgas_dict, project, "Nb_Orga")

On peut aussi compter le nombre d'organisations privées / publiques comme features

In [169]:
project.head(3)

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,...,rcn,grantDoi,Community,Degree,Degree_Centrality,New_Community,New_Community_Name,Nb_Publication,Project_Duration,Nb_Orga
0,817296,eDCaseMAN,CLOSED,"Defendable legal cases through an affordable, ...",2018-05-01,2018-09-30,71429,50000,H2020-EU.2.3.,EIC-SMEInst-2018-2020,...,217713,10.3030/817296,0,0,0.000000,-99,16,0,152.0,1
1,764717,WinWind,CLOSED,Winning social acceptance for wind energy in w...,2017-10-01,2020-03-31,"2124462,5","2124462,5",H2020-EU.3.3.,LCE-21-2017,...,211548,10.3030/764717,1,427,0.012069,1,0,4,912.0,12
2,784994,PentaHelix,CLOSED,Multi stakeholder and governance approach for ...,2018-03-01,2021-09-30,"1808343,75","1808343,75",H2020-EU.3.3.,EE-09-2016-2017,...,213566,10.3030/784994,1,74,0.002092,1,0,0,1309.0,11
